# ABS Quarterly Estimated Resident Population 3101

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Plot</a></span><ul class="toc-item"><li><span><a href="#National-and-State-Populations" data-toc-modified-id="National-and-State-Populations-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>National and State Populations</a></span></li></ul></li><li><span><a href="#Finished" data-toc-modified-id="Finished-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# local imports
import decompose
from abs_data_capture import (
    get_fs_constants,
    get_meta_constants,
    get_plot_constants,
    get_ABS_meta_and_data,
    find_id,
)
from plotting import (
    line_plot,
    set_chart_dir,
    recalibrate,
    clear_chart_dir,
    plot_covid_recovery,
    calc_growth,
    plot_growth_finalise,
    seas_trend_plot,
    abbreviate,
)

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# Display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
catalogue_id = "3101"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)

erp_dict = get_ABS_meta_and_data(catalogue_id)
if erp_dict is None:
    sys.exit(-1)

erp_meta = erp_dict[META_DATA]
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()
RECENT, plot_times, plot_tags = get_plot_constants(erp_meta)

Found URL for a ZIP file on ABS web page
Retrieving zip-file from cache ...
Extracting DataFrames from the zip-file ...


## Plotting

In [3]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Deaths

In [4]:
table = "1"
data = erp_dict[table]
rows = erp_meta[erp_meta[did_col].str.contains("[Dd]eaths")]
for index, row in rows.iterrows():
    id, units = row[id_col], row[unit_col]
    series, units = recalibrate(data[id], units)
    title = row[did_col].replace(" ;", "")

    line_plot(
        series,
        title=title,
        ylabel=units,
        rfooter=f"{source} {table}",
        lfooter=f"{row[type_col]} series. ",
        show=SHOW,
    )

    decomposed = decompose.decompose(
        series,
        constantSeasonal=True,
        # seasonalSmoother=decompose.s3x9,
    )
    seas_trend_plot(
        decomposed[["Seasonally Adjusted", "Trend"]],
        starts=RECENT,
        tags="Mark",
        title=title,
        ylabel=units,
        rfooter=f"{source} {table}",
        lfooter="Seasonally adjusted using in-house methods",
        show=SHOW,
    )

    result = seasonal_decompose(series, model="multiplicable", period=4)
    df = pd.DataFrame(
        [series / result.seasonal, result.trend],
        index=["Seasonally Adjusted", "Trend"],
    ).T
    seas_trend_plot(
        df,
        starts=RECENT,
        tags="Python",
        title=title,
        ylabel=units,
        rfooter=f"{source} {table}",
        lfooter="Seasonally adjusted using Python's seasonal_decompose() from statsmodels.tsa.seasonal",
        show=SHOW,
    )

### National and State Populations

In [5]:
table = "4"
erp_df = erp_dict[table]
erp_phrase = "Estimated Resident Population ;  Persons ;"

states = (
    erp_meta.loc[
        (erp_meta[table_col] == table) & (erp_meta[did_col].str.contains(erp_phrase)),
        did_col,
    ]
    .str.replace(erp_phrase, "")
    .str.replace(" ;", "")
    .str.strip()
    .str.strip()
    .to_list()
)

for state in states:
    # get relevant data series
    selector = {
        table: table_col,
        erp_phrase: did_col,
        f";  {state} ;": did_col,  # Australia, South Australia, etc.
    }
    id, units = find_id(erp_meta, selector, verbose=False)
    series = erp_df[id]
    series.name = "Estimated Resident Population"
    units = "Number Persons" if units == "Persons" else units
    series, units = recalibrate(series, units)

    # plot complete series
    title = f"Estimated Resident Population: {abbreviate(state)}"
    line_plot(
        series,
        title=title,
        ylabel=units,
        rfooter=f"{source} {table}",
        show=SHOW,
    )

    plot_covid_recovery(
        series,
        title=title,
        ylabel=units,
        tags="-covid",
        rfooter=f"{source} {table}",
        show=SHOW,
    )

    for start, tag in zip(plot_times, plot_tags):
        if state != "Australia" and start is None:
            continue

        growth_percent = calc_growth(series)
        plot_growth_finalise(
            *growth_percent,
            from_=start,
            tag=f"percent-{tag}",
            title=f"Growth in the {title}",
            rfooter=f"{source} {table}",
            show=SHOW,
        )

        growth_number = pd.DataFrame(
            [series.diff(4), series.diff(1)], index=["Annual", "Quarterly"]
        ).T
        growth_number, gunits = recalibrate(growth_number, units)
        plot_growth_finalise(
            growth_number["Annual"],
            growth_number["Quarterly"],
            from_=start,
            ylabel=gunits,
            title=f"Growth in the {title}",
            tag=f"numeric-{tag}",
            rfooter=f"{source} {table}",
            show=SHOW,
        )

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Mon May 01 2023 11:53:33

Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.13.1

sys       : 3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]
matplotlib: 3.7.1
pandas    : 2.0.1

Watermark: 2.3.1



In [7]:
print("Finished")

Finished
